## Functions with Multiple Parameters vs. Curried Functions

We will first study this in Scala. 
  - Currying named after mathematician/computer scientist Haskell Curry.
  - Multiple argument functions can be seen as a sequence of single argument functions.
  
  

In [ ]:
def addTwo (x: Int, y: Int): Int = x + y

Note the type of addTwo is `(Int, Int) => Int`

In [ ]:
// Scala supports curried functions: sort of.

def curriedAddTwo (x: Int) (y: Int) = x + y 



Note the type of curriedAddTwo is `Int => (Int => Int)`.

  - If you give it a number, it returns a function from number to number.
  

### Question 1.

Interpret the meaning of the call `curriedAddTwo (15)`.


In [ ]:
val f1 = curriedAddTwo(15)
// Oops.. scala does not allow this syntax although lettuce does.

In [ ]:
val f1 = curriedAddTwo (15)(_)
// f1 has plugged in x but y has not be set. Remember _ makes it an anonymous function
val x = f1(20)
val y = f1(30)
val z = f1(45)

In [ ]:
// Currying in anonymous function syntax
val curriedAddTwoAnon = (x: Int) => { (y:Int) => (x + y) } 

In [ ]:
val f1 = curriedAddTwoAnon(15) // Now this is closer to lettuce syntax
val f2 = curriedAddTwoAnon(15)(_) // This is also OK

### Extending Lettuce to Multiple Argument Functions

In class, we have explored just single argument functions in lettuce. In this problem we will 
explore multiple argument functions.

Consider: 

~~~
let foo1 = function (x, y)
            x - 2 * y
          in 
       foo1(10, 15)
~~~

this code should return -20

We will allow zero arguments as well.

~~~
let x = 5 in 
let bar1 = function()
           x 
           in 
    bar1()
~~~

This code should return 5.

Consider the grammar specification we have seen thus far.

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & FunDef( \mathbf{Identifier}^*, \mathbf{Expr}) & \text{Note multiple parameters possible} \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}^*) & \text{function call - expr(expr1, ... , exprn)} \\
 & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
\end{array}$$

Write the scala definition for `FunDef` and `FunCall`. Please use lists to implement the Kleene Star.

In [ ]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
//BEGIN SOLUTION
case class FunDef(idList: List[String], e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: List[Expr]) extends Expr
//END SOLUTION
case class TopLevel(e: Expr) extends Program

We will now redefine closures for functions with zero or more args. When our function had one argument, our closures were defined as `Closure(id, expr, env)`. We would now like to define closures as
`Closure([id1, ..., idn], expr, env)` where 
  - `id1, ..., idn` are the arguments for the function to be called. 
  - `expr` is the body of the function and 
  - `env` is the stored environment for static scoping.
 
 $$\begin{array}{rcl}
 \mathbf{Value} & \Rightarrow & num(\mathbf{Double}) \\
 & \Rightarrow & closure(\mathbf{String}^*, \mathbf{Expr}, \mathbf{Environment}) \\
 & \Rightarrow & error \\
 \end{array}$$
 
 For __Environment__ please use a scala immutable map from __String__ to __Value__

In [ ]:
sealed trait Value
case class Num(d: Double) extends Value
case object Error extends Value
//BEGIN SOLUTION
case class Closure(s: List[String], e: Expr, env: Map[String, Value]) extends Value
//END SOLUTION

Build an interpreter using the following semantic rules. Ensure that your interpreter correctly deals with the cases that give rise to error by throwing an exception.

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array} (\text{#3})} $$
$$\newcommand\eval{\mathbf{eval}}$$
$$\semRule{}{\eval(\texttt{FunDef([id1,..., idk], e)},\sigma) = \text{Closure}(\texttt{[id1,..., idk]}, \texttt{e}, \sigma)}{fundef}$$

$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n = k},\ (\forall\ i \in \{ 1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = v_i,  v_i \not= \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \eval(\texttt{fBody}, \color{red}{\sigma_{cl} \circ [id1 \rightarrow v_1, \ldots, idk \rightarrow v_k]})}{funcall-ok}$$
           
Let us interpret this rule. Read the points below carefully.
   - __Purpose__ : Evaluate an expression of the form `FunCall(e, [e1,...,ek])` where `e` is the expr for the called function, and `e1, ..., ek` are exprs for the arguments of this call. There are $k$ arguments.
     - (A) Evaluating `e` must yield a closure of the form $\text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}})$.
     - (B) The number of arguments for the closure $n$ must equal that of the function call $k$.
     - (C) Evaluating each of the $k$ arguments `ei` for $i = 1, \ldots, k$ must yield $v_i$ where $v_i$ is not error.
     - (D) Then the result of evaluating the call is the same as that of evaluating `fBody` under the environment $\color{red}{\sigma_{cl}}$ extended by mapping the formal parameters `id1.., idk` to $v_1, \ldots, v_k$, respectively.


We can write some error rules.

$$\semRule {\eval(\texttt{e}, \sigma) \not\in \text{Closure}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-not-a-function}$$


$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n \not= k}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-wrong-num-args}$$
           
  
$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n = k},\ (\exists\ i \in \{1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-arg-error}$$
           

In [ ]:
class ErrorException(s:String) extends Exception(s){}  


def valueToNumber(v: Value): Double = v match {
    case Num(d) => d
    case _ => throw new ErrorException(s"Could not convert value $v to a number")
}

def eval(e: Expr, env: Map[String, Value]): Value = {
    def addValues(v1: Value, v2: Value): Value = 
        Num ( valueToNumber(v1) + valueToNumber(v2) )
    
    e match {
        case Const(d) => Num(d)
        case Ident(x) => {
            //BEGIN SOLUTION
            if (env contains x) {
                env(x)
            } else {
                throw new ErrorException(s"Could not find identifier $x")
            }
            //END SOLUTION
        }
        case Plus(e1,e2) => {
            val v1 = eval(e1, env)
            val v2 = eval(e2, env)
            addValues(v1,v2)
        }
        case Let(id, e1, e2) => {
            val v1 = eval(e1, env)
            val env2 = env + (id -> v1)
            eval(e2, env2)
        }
        
        case FunDef(idList, e) => {
            //BEGIN SOLUTION
            Closure(idList, e, env)
            //END SOLUTION
        }
        
        case FunCall(e, eList) => {
            //BEGIN SOLUTION
            val v = eval(e, env)
            v match {
                case Closure(argList, fBody, closureEnv) => {
                    // Check that the length of the list of arguments
                    // matches with the length of the list of formal parameters
                    // in the closure
                    if (argList.length == eList.length){
                        // The two lists are of the same length
                        // Recursively evaluate the list of arguments
                        val valList = eList.map {eval(_, env)}
                        // Combine argument list with the list of values
                        val zipList = argList.zip(valList)
                        // Simply update closureEnv
                        val newEnv = closureEnv ++ zipList
                        // Call the body of the function with new environment
                        eval(fBody, newEnv)
                    } else {
                        throw new ErrorException(s"Argument length mismatch")
                    }
                }
                case _ => throw new ErrorException(s"Tried to use a numerical value as a function")
            }
            //END SOLUTION
        }
    }
}


def evalProgram(p: Program): Value = p match {
    case TopLevel(e) => try
            eval(e, Map.empty)
    catch {
        case e: ErrorException => {
            println(e)
            Error
        }
        case e: IllegalArgumentException => {
            println(e)
            Error
        }
        case e => {
            println("Unknown Exception " + e.toString)
            Error
        }
            
    }
}

In [ ]:
val p1 = TopLevel(Const(5.0))
val p2 = TopLevel(Plus(Const(4.0), Const(5.0)))
val p3 = TopLevel(Let("x", Const(1.0), Plus(Ident("x"), Const(5.0))))

assert(evalProgram(p1) == Num(5.0), "Test 1")
assert(evalProgram(p2) == Num(9.0), "Test 2")
assert(evalProgram(p3) == Num(6.0), "Test 3")

In [ ]:
val p4 = TopLevel(
    Let(
        "f", 
        FunDef(List("x", "y", "z"), Plus(Ident("x"), Ident("z"))), 
        FunCall(Ident("f"), List(Const(1.0), Const(2.0), Const(3.0)))
    )
)
assert(evalProgram(p4) == Num(4.0), "Test 4") // (funcall-ok)

In [ ]:
val p5 = TopLevel(
    Let(
        "f", 
        Const(2.0), 
        FunCall(Ident("f"), List(Const(1.0), Const(2.0), Const(3.0)))
    )
)
assert(evalProgram(p5) == Error, "Test 5") // (funcall-not-a-function)

In [ ]:
val p6 = TopLevel(
    Let(
        "f", 
        FunDef(List("x", "y", "z"), Plus(Ident("x"), Ident("z"))), 
        FunCall(Ident("f"), List(Const(1.0)))
    )
)
assert(evalProgram(p6) == Error, "Test 6") // (funcall-wrong-num-args)

In [ ]:
val p7 = TopLevel(
    Let(
        "f", 
        FunDef(List("x", "y", "z"), Plus(Ident("x"), Ident("z"))), 
        FunCall(Ident("f"), List(Const(1.0), Const(2.0), Ident("giraffe")))
    )
)
assert(evalProgram(p7) == Error, "Test 7") // (funcall-arg-error)